In [1]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'hw3'))
	print(os.getcwd())
except:
	pass

c:\Users\Renjie Zhu\Documents\workspace\recommender_system\hw3


# Homework 3 - Renjie Zhu - A53266114

In [2]:
import random
import numpy as np
import scipy
import scipy.optimize

In [3]:
import gzip
from collections import defaultdict

def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')

def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [4]:
bookCount = defaultdict(int)
bookSet = set()
totalRead = 0
user_read = defaultdict(set)
data = []

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    bookSet.add(book)
    totalRead += 1
    user_read[user].add(book)
    data.append((user, book))

## 1.



In [5]:
training = data[:190000]
validation = data[190000:]
validation_new = []

In [6]:
for user, book in validation:
    sample = random.sample(bookSet.difference(user_read[user]), 1)[0]
    validation_new.append([user, book, 1])
    validation_new.append([user, sample, 0])
validation = np.array(validation_new)

In [7]:
book_user = defaultdict(set)
user_read = defaultdict(set)

for user, book in training:
    book_user[book].add(user)
    user_read[user].add(book)

In [8]:
# baseline model
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

popular = set()
count = 0
for ic, i in mostPopular:
    count += ic
    popular.add(i)
    if count > totalRead * 0.58: break

In [9]:
pred = []
for _, book, _ in validation:
    if book in popular:
        pred.append(1)
    else:
        pred.append(0)


In [10]:
valid_gt = validation[:,2].astype(int)

In [11]:
print(f"Baseline accuracy: {sum(pred==valid_gt)/len(pred)}.")

Baseline accuracy: 0.65765.


## 2.

Better threshold

In [12]:
# better threshold
percentile = list(range(40, 60, 1))
popularSet = []
for p in percentile:
    popular = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        popular.add(i)
        if count > totalRead * (p/100): break
    popularSet.append(popular)

    pred = []
    for _, book, _ in validation:
        if book in popular:
            pred.append(1)
        else:
            pred.append(0)
    
    print(f"Baseline accuracy: {sum(pred==valid_gt)/len(pred)}; threshold: {p} th")

Baseline accuracy: 0.63875; threshold: 40 th
Baseline accuracy: 0.64015; threshold: 41 th
Baseline accuracy: 0.6419; threshold: 42 th
Baseline accuracy: 0.6447; threshold: 43 th
Baseline accuracy: 0.6455; threshold: 44 th
Baseline accuracy: 0.646; threshold: 45 th
Baseline accuracy: 0.6477; threshold: 46 th
Baseline accuracy: 0.649; threshold: 47 th
Baseline accuracy: 0.6502; threshold: 48 th
Baseline accuracy: 0.6521; threshold: 49 th
Baseline accuracy: 0.6532; threshold: 50 th
Baseline accuracy: 0.65325; threshold: 51 th
Baseline accuracy: 0.6549; threshold: 52 th
Baseline accuracy: 0.6562; threshold: 53 th
Baseline accuracy: 0.65615; threshold: 54 th
Baseline accuracy: 0.6561; threshold: 55 th
Baseline accuracy: 0.6574; threshold: 56 th
Baseline accuracy: 0.6582; threshold: 57 th
Baseline accuracy: 0.65765; threshold: 58 th
Baseline accuracy: 0.6571; threshold: 59 th


The better threshold is 57th percentage from the above test.

## 3.

Jaccard similarity


In [13]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

def Max_Jaccard(user, book):
    similarities = []
    b = book_user[book]
    for read in user_read[user]:
        similarities.append(Jaccard(b, book_user[read]))
    
    return max(similarities)

In [14]:
ps = np.linspace(0.005, 0.015, 11)
for p in ps:
    pred = []
    for user, book, _ in validation:
        sim = Max_Jaccard(user, book)
        if sim > p:
            pred.append(1)
        else:
            pred.append(0)

    print(f"Jaccard accuracy: {sum(pred==valid_gt)/len(pred)}; threshold: {p}")

Jaccard accuracy: 0.5996; threshold: 0.005
Jaccard accuracy: 0.6064; threshold: 0.006
Jaccard accuracy: 0.6124; threshold: 0.006999999999999999
Jaccard accuracy: 0.6186; threshold: 0.008
Jaccard accuracy: 0.62345; threshold: 0.009
Jaccard accuracy: 0.6255; threshold: 0.009999999999999998
Jaccard accuracy: 0.62565; threshold: 0.011
Jaccard accuracy: 0.62445; threshold: 0.011999999999999999
Jaccard accuracy: 0.6232; threshold: 0.012999999999999998
Jaccard accuracy: 0.61825; threshold: 0.013999999999999999
Jaccard accuracy: 0.61125; threshold: 0.015


With above test, the best the Jaccard accuracy can do is about a 62.645% accuracy with threshold of 0.011.

## 4.

Classifier with population and Jaccard accuracy

In [15]:
result = []
for p in range(58,65,1):
    for t in np.linspace(0.004,0.010, 7):
        popular = set()
        count = 0
        for ic, i in mostPopular:
            count += ic
            popular.add(i)
            if count > totalRead * (p/100): break

        pred = []
        for user, book, _ in validation:
            sim = Max_Jaccard(user, book)
            if (sim > t) and (book in popular):
                pred.append(1)
            else:
                pred.append(0)

        print(f"Combined accuracy: {sum(pred==valid_gt)/len(pred)} pop ({p}), thr ({t}).")
        result.append((sum(pred==valid_gt)/len(pred),p,t))
result.sort()
print(result[-1])

Combined accuracy: 0.66425 pop (58), thr (0.004).
Combined accuracy: 0.6641 pop (58), thr (0.005).
Combined accuracy: 0.66485 pop (58), thr (0.006).
Combined accuracy: 0.66465 pop (58), thr (0.007).
Combined accuracy: 0.6646 pop (58), thr (0.008).
Combined accuracy: 0.6639 pop (58), thr (0.009000000000000001).
Combined accuracy: 0.6603 pop (58), thr (0.01).
Combined accuracy: 0.6641 pop (59), thr (0.004).
Combined accuracy: 0.664 pop (59), thr (0.005).
Combined accuracy: 0.66475 pop (59), thr (0.006).
Combined accuracy: 0.6647 pop (59), thr (0.007).
Combined accuracy: 0.6645 pop (59), thr (0.008).
Combined accuracy: 0.66395 pop (59), thr (0.009000000000000001).
Combined accuracy: 0.6603 pop (59), thr (0.01).
Combined accuracy: 0.66485 pop (60), thr (0.004).
Combined accuracy: 0.66495 pop (60), thr (0.005).
Combined accuracy: 0.66565 pop (60), thr (0.006).
Combined accuracy: 0.6657 pop (60), thr (0.007).
Combined accuracy: 0.6656 pop (60), thr (0.008).
Combined accuracy: 0.66515 pop (60

In [16]:
f = open("predictions_Read.txt", 'w')
popular = set()
count = 0
for ic, i in mostPopular:
    count += ic
    popular.add(i)
    if count > totalRead * (64/100): break

for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        #header
        f.write(l)
        continue
    user,book = l.strip().split('-')
    sim = Max_Jaccard(user, book)
    if (sim > 0.008) and (book in popular):
        f.write(user + '-' + book + ",1\n")
    else:
        f.write(user + '-' + book + ",0\n")

f.close()

The Kaggle username is "Renjie Zhu".

## 9.

Rating prediction

In [17]:
user_rating = []
# users = []
# items = []
for user,book,rating in readCSV("train_Interactions.csv.gz"):
    user_rating.append((user, book, int(rating)))
    # users.append(user)
    # items.append(book)

training = user_rating[:190000]
validation = user_rating[190000:]

In [18]:
alpha = sum([int(r) for _,_,r in training]) / len(training)
userBiases = defaultdict(float)
itemBiases = defaultdict(float)
book_user = defaultdict(list)
user_book = defaultdict(list)

for u,b,r in training:
    book_user[u].append((u,b))
    user_book[b].append((u,b))


nUsers = len(book_user)
nItems = len(user_book)
users = list(book_user.keys())
items = list(user_book.keys())


In [19]:
def prediction(user, item):
    try:
        return alpha + userBiases[user] + itemBiases[item]
    except:
        return alpha

In [20]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [21]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(u, b) for u,b,_ in training]
    cost = MSE(predictions, labels)
    # print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost


In [22]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(training)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for u,b,r in training:
        pred = prediction(u, b)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[b] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return np.array(dtheta)

In [23]:
labels = [r for _,_,r in training]

In [24]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, 1))

(array([ 3.89709313e+00, -9.83268551e-06,  5.26797138e-05, ...,
         5.41367671e-07, -8.90170173e-06, -2.46909977e-05]),
 1.4734687158357118,
 {'grad': array([-2.47033492e-08, -3.42983207e-09, -1.04124480e-08, ...,
         -2.83062290e-10, -1.05817276e-09, -1.37158955e-09]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 4,
  'nit': 2,
  'warnflag': 0})

In [25]:
# prediction on validation set
pred=[]
real=[]
for u,b,r in validation:
    pred.append(prediction(u,b))
    real.append(r)

print(f"The MSE on validation set is {MSE(pred,real)}.")

The MSE on validation set is 1.490780087874981.


## 10.

In [26]:
print(f"User: largest {max(userBiases, key=userBiases.get)} ; smallest {min(userBiases, key=userBiases.get)}.")
print(f"Book: largest {max(itemBiases, key=itemBiases.get)} ; smallest {min(itemBiases, key=itemBiases.get)}.")

User: largest u92864068 ; smallest u11591742.
Book: largest b76915592 ; smallest b57299824.


## 11.

In [27]:
lamb = [1e-6,1e-5,1e-4,1e-3,1e-2]
for l in lamb:
    scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, l))

    pred=[]
    real=[]
    for u,b,r in validation:
        pred.append(prediction(u,b))
        real.append(r)

    print(f"The MSE on validation set is {MSE(pred,real)} with lambda = {l}.")

The MSE on validation set is 1.1261017419441643 with lambda = 1e-06.
The MSE on validation set is 1.1104914253254137 with lambda = 1e-05.
The MSE on validation set is 1.1774533947047277 with lambda = 0.0001.
The MSE on validation set is 1.3981533564653221 with lambda = 0.001.
The MSE on validation set is 1.4785348032993548 with lambda = 0.01.


In [28]:
scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(nUsers+nItems),
                             derivative, args = (labels, 1e-5))

(array([ 3.80834896e+00, -5.11998154e-04,  2.61209893e-01, ...,
        -3.35722196e-02, -3.80834971e-01, -7.70535389e-01]),
 0.960966738376091,
 {'grad': array([-9.34277656e-06, -6.08890426e-08, -1.13360640e-07, ...,
          2.81785196e-07,  3.77359143e-07,  4.27804254e-07]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 153,
  'nit': 121,
  'warnflag': 0})

In [29]:
f = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        f.write(l)
        continue
    u,b = l.strip().split('-')
    f.write(u + '-' + b + ',' + str(prediction(u,b)) + '\n')


f.close()

The Kaggle username is "Renjie Zhu".